In [3]:
import numpy as np

In [ ]:
def Option_binomial(
    S0, K, r, sigma, N, is_call=True, exercise_dates=None
):
    """
    El código Evalúa una opción tipo Bermuda con un modelo binomial.

    Parámetros:
    - S0: precio inicial del subyacente
    - K: strike
    - r: tasa libre de riesgo anual
    - sigma: volatilidad anual
    - N: Días de evaluación
    - is_call: True si es call, False si es put
    - exercise_dates: lista de pasos (0...N) donde se permite ejercer la opción en caso de ser de tipo bermuda, 
                      si la lista contiene todos los días el resultado será la evaluación de una opción americana, 
                      y si no se agregan días (), se obtiene la valuación de una opción europea.
    """
    dt = 1/365
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    r = np.log(1+r*N/360)*(365/N) # Composición continua
    p = (np.exp(r * dt) - d) / (u - d)
    discount = np.exp(-r * dt)

    # Inicializa árboles
    stock_tree = np.zeros((N + 1, N + 1))
    option_tree = np.zeros((N + 1, N + 1))

    # Llenar árbol de precios del subyacente
    for i in range(N + 1):
        for j in range(i + 1):
            stock_tree[j, i] = S0 * (u ** (i - j)) * (d ** j)

    # Valor terminal
    if is_call:
        option_tree[:, N] = np.maximum(stock_tree[:, N] - K, 0)
    else:
        option_tree[:, N] = np.maximum(K - stock_tree[:, N], 0)

    # Si no se pasan fechas, se comporta como opción europea
    if exercise_dates is None:
        exercise_dates = [N]

    # propagación hacia atrás
    for i in range(N - 1, -1, -1):
        for j in range(i + 1):
            continuation = discount * (
                p * option_tree[j, i + 1] + (1 - p) * option_tree[j + 1, i + 1]
            )
            if i in exercise_dates:
                intrinsic = max(
                    stock_tree[j, i] - K, 0
                ) if is_call else max(K - stock_tree[j, i], 0)
                option_tree[j, i] = max(continuation, intrinsic)
            else:
                option_tree[j, i] = continuation

    return option_tree[0, 0]

In [ ]:
# Parámetros de la opción call
S0 = 44.36      # Precio inicial del subyacente
K = 41.25       # Strike
r = 0.08565002  # Tasa libre de riesgo anual   
sigma = 0.33823359     # Volatilidad anual
N = 112         # Pasos del árbol binomial
is_call = True  # Call

# Fechas de ejercicio permitidas 
exercise_steps = np.linspace(28,112,4)

# Valuación
precio = Option_binomial(S0, K, r, sigma, N, is_call = True, exercise_dates = exercise_steps)
print(f"Valor de la opción Bermuda call: {precio:.4f}")

Valor de la opción Bermuda call: 5.6699


In [ ]:
# Parámetros de la opción put
S0 = 44.36      # Precio inicial del subyacente
K = 46.58     # Strike
r = 0.08565002  # Tasa libre de riesgo anual
sigma = 0.33823359     # Volatilidad anual
N = 112         # Pasos del árbol binomial
is_call = True  # Call

# Fechas de ejercicio permitidas (por ejemplo, cada trimestre)
exercise_steps = np.linspace(28,112,4)

# Valuación
precio = Option_binomial(S0, K, r, sigma, N, is_call = False, exercise_dates= exercise_steps)
print(f"Valor de la opción Bermuda put: {precio:.4f}")

Valor de la opción Bermuda put: 4.0184
